In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib agg

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
       
from HPIB.HP4155 import HP4155
from HPIB.HPT import Plot, PlotVgs, PlotVp, CalcIsSat, SecDer, Plot2P, PlotDiode4P,WriteLog
from HPIB.DevParams import UMC
from HPIB.INOSerial import Arduino

from BFmodule import DR

from YFunc import YFuncExtraction
#from Test import TestDevice, WriteLog

from IPython.display import clear_output, display
from os import makedirs, rename
from os.path import isfile
from time import sleep
from datetime import datetime, timedelta

Elsa=DR()

HP=HP4155("GPIB0::17", debug=False)

SM=Arduino("COM3")
print(SM.ask('*'.encode()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Opening API at 143.106.18.130
Elsa v2.4.2
HEWLETT-PACKARD,4155A,0,01.04:01.04:01.00
Comm Er


AttributeError: 'Arduino' object has no attribute 'inst'

In [ ]:
prepath = "C:/Users/Zucchi/Documents/Medidas/250424 Nb/"
makedirs(prepath, exist_ok=True)

In [ ]:
def Measure4P(savepath, I=1e-3, Im='SMU1',Ip='SMU2', Vm='SMU4', Vp='SMU3'):
    global HP, SM, Elsa
    HP.Samp4P(I=I, Im=Im, Ip=Ip, Vm=Vm, Vp=Vp, points=10, interval=3e-3)
    
    now=datetime.now().strftime('%y%m%d %H%M%S')
    name=HP.SingleSave(f"{savepath}/4P {Im[-1]}{Ip[-1]}{Vp[-1]}{Vm[-1]}p - {now}.csv", timeout=30)
    R=[]
    
    try:
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(name, name.replace('-', f'- {temp} -'))
        name=name.replace('-', f'- {temp} -')
    except Exception as err:
        print (">>> Error:", err)
    
    df=pd.read_csv(name, header=[0, 1]) 
    R+=[np.average(df['Vf']['None'].to_numpy()/df['If']['None'].to_numpy())]
    
    
    HP.SetSMU(Ip, 'VIp', 'Ip', 'I', 'CONS', Value=-1e-3, Comp=2)
    now=datetime.now().strftime('%y%m%d %H%M%S')
            
    name=HP.SingleSave(f"{savepath}/4P {Im[-1]}{Ip[-1]}{Vp[-1]}{Vm[-1]}m - {now}.csv", timeout=30)
    try:
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(name, name.replace('-', f'- {temp} -'))
        name=name.replace('-', f'- {temp} -')
    except Exception as err:
        print (">>> Error:", err)
    df=pd.read_csv(name, header=[0, 1])
    R+=[np.average(df['Vf']['None'].to_numpy()/df['If']['None'].to_numpy())]
    print(R)
    return R
    
def MeasureResistivity(savepath, I=1e-3):
    makedirs(savepath, exist_ok=True)
    R=[]
    
    R+=Measure4P(savepath, I=1e-3, Im='SMU1',Ip='SMU2', Vm='SMU4', Vp='SMU3')
    R+=Measure4P(savepath, I=1e-3, Im='SMU2',Ip='SMU3', Vm='SMU1', Vp='SMU4')
    R+=Measure4P(savepath, I=1e-3, Im='SMU3',Ip='SMU4', Vm='SMU2', Vp='SMU1')
    R+=Measure4P(savepath, I=1e-3, Im='SMU4',Ip='SMU1', Vm='SMU3', Vp='SMU2')

    return np.average(R), np.std(R), R

In [ ]:
SM.opench(1)
MeasureResistivity(f"{prepath}")

In [ ]:
 current_temp=Elsa.GetT('t4k')

In [ ]:
SM.opench(1)

###### Warmup measurement

MsrNo=1 #??? n tem tipo só uma medida nesse caso?

current_temp=np.around(Elsa.GetT('t4k')) #temperatura da elza 


direction="cooldown"
freq="Fine"
finish_temp=5
freq_temp=1 #A cada quantos kelvin ele vai medir

if not isfile(f"{prepath}/RxT {direction}.log"):
    with open(f"{prepath}/RxT {direction}.log", 'w') as MyFile:
        MyFile.write(f"Temp,ro,R,R1p,R1m,R2p,R2m,R3p,R3m,R4p,R4m\n")

HP.IntTime='LONG' #TEMPO DE INTEGRAÇÃO 
HP.LongNPLC=2   #QUANTOS CICLOS DA REDE ELE VAI ADOTAR COMO TEMPO DE INTEGRAÇÃO (120HZ)

#INTEGRAÇÃO PRA FAZER O VALOR MEDIO DAS MEDIDAS 

clear_output()

prog_bar=display('',display_id=True) #FUNÇÃO DE ATUALIZAÇÃO DO PYTHON, POSSO ATUALIZAR NESSA LINHA A MEDIDA

last_temp=current_temp

while (last_temp >= finish_temp):
    #loop until temp changes and is a multiple of freq_temp
    start=datetime.now()
    while not (np.around(current_temp)!=last_temp or current_temp%5!= 0):
        #verifica se está na temperatura correta (25 20 15) e coisa e tal ae ele sai do loop
        if (datetime.now()-start).seconds/60 > 120:
            break
        for i in range(12):
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K ({format(last_temp, '.1f')})")
            sleep(1)
    
        current_temp=Elsa.GetT('t4k')

    last_temp=np.around(current_temp)

    #aqui ele ja ta saindo a temperatura na base 5 
    
    for Msr in range(MsrNo): #considerando uma medida por temperatura faça:
        plt.close('all')
        clear_output()
        prog_bar=display('Measuring',display_id=True) #cria uma nova aba de progressão de medida 

        start=datetime.now()
        WriteLog(f"# {current_temp} K - Measurement {Msr+1} - {datetime.now().strftime('%y%m%d %H%M')}\n", f'{prepath}/{direction}/log.txt')
        SM.opench(1)

        RAvg, StDevR, Rarray=MeasureResistivity(f"{prepath}/{direction}/{freq}")

        ro=(np.pi/np.log(2))*100e-9*RAvg

        with open(f"{prepath}/RxT {direction}.log", 'a') as MyFile:
            MyFile.write(f"{format(last_temp, '07.3f')},{format(ro, '.5e')},{format(RAvg, '.5e')},{','.join([format(x, '.5e') for x in Rarray])}\n")

    
        WriteLog(f"{current_temp} K - Measurement {Msr+1} end . Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", f'{prepath}/{direction}/log.txt')

    try:
        current_temp=Elsa.GetT('t4k')
    except:
        pass
        if i < MsrNo:
            start=datetime.now()
            while(datetime.now()-start).seconds < MsrWait*60:
                prog_bar.update(f"Waiting: {int(MsrWait*60)-(datetime.now()-start).seconds} s")
                sleep(1)
prog_bar.update("Ramp measurement done")

In [ ]:
###### Warmup measurement

MsrNo=1 #??? n tem tipo só uma medida nesse caso?

current_temp=Elsa.GetT('t4k') #temperatura da elza 


direction="Warmup"
freq="Coarse"
finish_temp=295
freq_temp=10 #A cada quantos kelvin ele vai medir

if not isfile(f"{prepath}/RxT {direction}.log"):
    with open(f"{prepath}/RxT {direction}.log", 'w') as MyFile:
        MyFile.write(f"Temp,ro,R,R1p,R1m,R2p,R2m,R3p,R3m,R4p,R4m\n")

HP.IntTime='LONG' #TEMPO DE INTEGRAÇÃO 
HP.LongNPLC=2   #QUANTOS CICLOS DA REDE ELE VAI ADOTAR COMO TEMPO DE INTEGRAÇÃO (120HZ)

#INTEGRAÇÃO PRA FAZER O VALOR MEDIO DAS MEDIDAS 

clear_output()

prog_bar=display('',display_id=True) #FUNÇÃO DE ATUALIZAÇÃO DO PYTHON, POSSO ATUALIZAR NESSA LINHA A MEDIDA

last_temp=current_temp
while (last_temp <= finish_temp):
    #loop until temp changes and is a multiple of freq_temp
    start=datetime.now()
    while not (np.around(current_temp)!=last_temp and np.around(current_temp)%freq_temp==5):
        #verifica se está na temperatura correta (25 20 15) e coisa e tal ae ele sai do loop
        if (datetime.now()-start).seconds/60 > 120:
            break
        for i in range(12):
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K")
            sleep(1)
    
        current_temp=Elsa.GetT('t4k')

    last_temp=np.around(current_temp)

    #aqui ele ja ta saindo a temperatura na base 5 
    
    for Msr in range(MsrNo): #considerando uma medida por temperatura faça:
        plt.close('all')
        clear_output()
        prog_bar=display('Measuring',display_id=True) #cria uma nova aba de progressão de medida 

        start=datetime.now()
        WriteLog(f"# {current_temp} K - Measurement {Msr+1} - {datetime.now().strftime('%y%m%d %H%M')}\n", f'{prepath}/{direction}/log.txt')
        SM.opench(1)

        RAvg, StDevR, Rarray=MeasureResistivity(f"{prepath}/{direction}/{freq}")

        ro=(np.pi/np.log(2))*100e-9*RAvg

        with open(f"{prepath}/RxT {direction}.log", 'a') as MyFile:
            MyFile.write(f"{format(last_temp, '07.3f')},{format(ro, '.5e')},{format(RAvg, '.5e')},{','.join([format(x, '.5e') for x in Rarray])}\n")

    
        WriteLog(f"{current_temp} K - Measurement {Msr+1} end . Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", f'{prepath}/{direction}/log.txt')

    try:
        current_temp=Elsa.GetT('t4k')
    except: # perguntar qq essa marda faz
        pass
        if i < MsrNo:
            start=datetime.now()
            while(datetime.now()-start).seconds < MsrWait*60:
                prog_bar.update(f"Waiting: {int(MsrWait*60)-(datetime.now()-start).seconds} s")
                sleep(1)
prog_bar.update("Ramp measurement done")

In [ ]:
 current_temp = np.around(Elsa.GetT('t4k'))
print(current_temp)

In [ ]:
    global HP, SM, Elsa
    HP.Samp4P(I=1e-3, Im='SMU1', Ip='SMU2', Vm='SMU4', Vp='SMU3', points=10, interval=3e-3)

In [ ]:
prepath = "C:/Users/Zucchi/Documents/Medidas/250517 Nb/"


In [ ]:
I=1e-3
Im='SMU1'
Ip='SMU2'
Vm='SMU4'
Vp='SMU3'
makedirs(prepath, exist_ok=True)


In [ ]:
global HP, SM, Elsa
HP.Samp4P(I=I, Im=Im, Ip=Ip, Vm=Vm, Vp=Vp, points=10, interval=3e-3)
now=datetime.now().strftime('%y%m%d %H%M%S')
        
name=HP.SingleSave(f"{prepath}/4P {Im[-1]}{Ip[-1]}{Vp[-1]}{Vm[-1]}p - {now}.csv", timeout=30)
print(name)
R=[]
try:
    temp=format(Elsa.GetT('t4k'), '07.3f')
    rename(name, name.replace('-', f'- {temp} -'))
    name=name.replace('-', f'- {temp} -')
except Exception as err:
    print (">>> Error:", err)
df=pd.read_csv(name, header=[0, 1]) 
R+=[np.average(df['Vf']['None'].to_numpy()/df['If']['None'].to_numpy())]
print(R)

In [ ]:
 def save_mari(self, timeout=2, real=False):
    if self.term=="0": return "Parameters not set"
    
    print(f"Measuring {self.term} ", end='')
    self.measure()
    
    Poll=self.PollDR(1, 1, timeout)
    
    if Poll:
        return "Operation stopped or timeout"
    if real:
        df=self.get_realdata()
    else:
        df=self.get_data()
    return df

In [ ]:
prepath = "C:/Users/Zucchi/Documents/Medidas/250521 Nb/"

In [ ]:
from lib_measurement import teste
Elsa, HP, SM = teste.connect()

In [ ]:

current_temp=np.around(Elsa.GetT('t4k'))
while(current_temp>finish_temp):
    if(current_temp>35):
        while current_temp == current_temp_ and current_temp%10!=0:
            sleep(1)
            current_temp_= no.round(Elsa.GetT('t4k'))
    
    else:
        while current_temp == current_temp_:
            sleep(1)
            current_temp_= np.around(Elsa.GetT('t4k'))
       
    Rm,Ri,R,T_=teste.MeasureResistivity(Elsa, HP, SM, prepath)
    Res.append(Rm)
    T.append(T_)    
        


In [ ]:

current_temp=Elsa.GetT('t4k')


In [ ]:
plt.plot(Res, T, marker='o')

# Adiciona rótulos aos eixos e título
plt.xlabel("Eixo X")
plt.ylabel("Eixo Y")
plt.title("Gráfico de Pontos")

# Exibe o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Res, T, marker='o')

# Adiciona rótulos aos eixos e título
plt.xlabel("Eixo X")
plt.ylabel("Eixo Y")
plt.title("Gráfico de Pontos")

# Exibe o gráfico
plt.show()


In [ ]:
tupla=[]
Res=[]
T=[]
finish_temp=5
Rm,Ri,R,T_=teste.MeasureResistivity(Elsa, HP, SM, prepath)
Res.append(Rm)
T.append(T_)
print(Res)
print(T)

In [16]:
HP.inst.close()
SM.inst.close()
del HP, SM
print("Comm closed")

AttributeError: 'Arduino' object has no attribute 'inst'

# A PARTIR DAQUI É DOCUMENTAÇÃO E HISTÓRICO DE CÓDIGOS (N MEXE ZUCHI)

## As libs que sempre precisa importar por algum motivo

In [1]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
       
from HPIB.HP4155 import HP4155
from HPIB.HPT import Plot, PlotVgs, PlotVp, CalcIsSat, SecDer, Plot2P, PlotDiode4P,WriteLog
from HPIB.DevParams import UMC
from HPIB.INOSerial import Arduino
from scipy.constants import e, k


from BFmodule import DR

from YFunc import YFuncExtraction
#from Test import TestDevice, WriteLog

from IPython.display import clear_output, display
from os import makedirs, rename
from os.path import isfile
from time import sleep
from datetime import datetime, timedelta

Elsa=DR()

HP=HP4155("GPIB0::17", debug=False)

SM=Arduino("COM3")
print(SM.ask('*'.encode()))

Opening API at 143.106.18.130
Elsa v2.4.2
HEWLETT-PACKARD,4155A,0,01.04:01.04:01.00
b'*'



## Abrindo porta para conexão da elza pela switch matrix

In [3]:
SM.opench(3)

'Open INO: ch 3'

## Configurando o hp

In [ ]:
self.SetSMU(Im, 'V1', 'I1')
#aqui eu defino 
# definição sem nome (self, SMUno, VNAME, INAME, Mode='COMM', Func='CONS', Comp='1e-3', SRES='0OHM', Value=0)

In [2]:
HP.Set4PSMU(-0.5, 1, 0.01, Im='SMU1', Ip='SMU2', Vm='SMU3', Vp='SMU4', Comp=2e-3)
    #self.SetSMU(Im, 'V1', 'I1')   Define como referencia da fonte de corrente 
    #self.SetSMU(Ip, 'V2', 'I2', 'I', 'VAR1', Comp=Comp)    Define a fonte de corrente como variavel
    #self.SetSMU(Vm, 'V3', 'I3', 'I', 'CONS', Value=0, Comp=Comp)  
    #self.SetSMU(Vp, 'V4', 'I4', 'I', 'CONS', Value=0, Comp=Comp)
    #self.SetVar('VAR1', 'I', Istart, Istop, (Istop-Istart)/(Points))



Set 4P


0